In [1]:
# Importing Necessary Libraries

import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import models
from keras.layers import Dense, Conv1D, Conv2D, Flatten, MaxPooling1D, MaxPooling2D
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
from keras.layers import Dropout

Using TensorFlow backend.


## TASK 1
In this problem, we are trying to remove the sound of eating chips from various sound signals using 1D CNN. CNN is very good at finding local patterns and hence can be used in sequence processing. We are given the following set of audio signals for training and we have to train a deep neural network, containing 1D CNN and fully connected layers, for denoising speech signals.  
#### Clean Training Signal
<audio src="data/train_clean_male.wav" controls>Audio file not availablle</audio>
#### Dirty Training Signal
<audio src="data/train_dirty_male.wav" controls>Audio file not availablle</audio>

In [2]:
# Loading the .wav files
# Ground Truth for training
s, sr=librosa.load(r'data\train_clean_male.wav', sr=None)
S=librosa.stft(s, n_fft=1024, hop_length=512)
# Unclean train file
sn, sr=librosa.load(r'data\train_dirty_male.wav', sr=None)
X=librosa.stft(sn, n_fft=1024, hop_length=512)
# Test 1
tn, sr=librosa.load(r'data\test_x_01.wav', sr=None)
A = librosa.stft(tn, n_fft=1024, hop_length=512)
# Test 2
t, sr=librosa.load(r'data\test_x_02.wav', sr=None)
B = librosa.stft(t, n_fft=1024, hop_length=512)

In [3]:
# Obtaining amplitude
y_data = np.abs(S).T
X_data = np.abs(X).T
# 90% of training data is used fr training and rest 10% for validation
X_train = X_data[:int(X_data.shape[0]*0.9)]
X_valid = X_data[int(X_data.shape[0]*0.9):]
y_train = y_data[:int(y_data.shape[0]*0.9)]
y_valid = y_data[int(y_data.shape[0]*0.9):]
# Test Data
X_test1 = np.abs(A).T
X_test2 = np.abs(B).T

In [4]:
# Expanding dimension of data
X_train = X_train[:,:,np.newaxis]
X_valid = X_valid[:,:,np.newaxis]
X_test1 = X_test1[:,:,np.newaxis]
X_test2 = X_test2[:,:,np.newaxis]

In [5]:
# Defining the model architecture
model = Sequential()
model.add(Conv1D(32, 7, input_shape = (513,1), padding='same', activation='relu', kernel_initializer='he_normal'))
model.add(MaxPooling1D(2))
model.add(Conv1D(64, 5, padding='same', activation='relu', kernel_initializer='he_normal'))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(1024, activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.3))
model.add(Dense(513, activation='relu', kernel_initializer='he_normal'))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
# Model Summary
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 513, 32)           256       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 256, 32)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 256, 64)           10304     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 128, 64)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              8389632   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
__________

In [7]:
# Compiling the model
model.compile(loss = 'MSE', optimizer = Adam(), metrics = ['accuracy'])

In [8]:
# Training the model on data
history = model.fit(X_train, y_train, epochs = 20, batch_size = 128, validation_data = (X_valid, y_valid))


Train on 2213 samples, validate on 246 samples
Epoch 1/20
2213/2213 [==============================] - 24s 11ms/step - loss: 0.0975 - acc: 0.0538 - val_loss: 0.0301 - val_acc: 0.0854
Epoch 2/20
2213/2213 [==============================] - 0s 160us/step - loss: 0.0533 - acc: 0.1405 - val_loss: 0.0240 - val_acc: 0.2154
Epoch 3/20
2213/2213 [==============================] - 0s 160us/step - loss: 0.0391 - acc: 0.2300 - val_loss: 0.0198 - val_acc: 0.2683
Epoch 4/20
2213/2213 [==============================] - 0s 157us/step - loss: 0.0314 - acc: 0.2815 - val_loss: 0.0178 - val_acc: 0.3618
Epoch 5/20
2213/2213 [==============================] - 0s 157us/step - loss: 0.0283 - acc: 0.3037 - val_loss: 0.0166 - val_acc: 0.3455
Epoch 6/20
2213/2213 [==============================] - 0s 158us/step - loss: 0.0262 - acc: 0.3186 - val_loss: 0.0149 - val_acc: 0.4024
Epoch 7/20
2213/2213 [==============================] - 0s 159us/step - loss: 0.0239 - acc: 0.3290 - val_loss: 0.0144 - val_acc: 0.3455


In [9]:
# Getting output prediction for test files
outt = model.predict(X_data[:,:,np.newaxis])
out1 = model.predict(X_test1)
out2 = model.predict(X_test2)

In [10]:
# Adding phase information
stestt = (X/np.abs(X))*outt.T
stest1 = (A/np.abs(A))*out1.T
stest2 = (B/np.abs(B))*out2.T

In [11]:
# SNR for training data
snr = 10*np.log10(np.add.reduce(np.square(S), axis = None)/np.add.reduce(np.square(S-stestt), axis = None))
print('SNR:', snr)

SNR: (17.173919677734375+4.238504469394684j)


In [12]:
# Performing Inverse Short Time Fourier Transform to get the output signal
sh_test1 = librosa.istft(stest1, hop_length=512)
sh_test2 = librosa.istft(stest2, hop_length=512)

In [13]:
# Storing the cleaned test files
librosa.output.write_wav(r'data\test_x_03_clean.wav', sh_test1, sr)
librosa.output.write_wav(r'data\test_x_04_clean.wav', sh_test2, sr)

The 1D CNN performs very well in denoising the speech signals. I didn't have to use too many layers to get a better output than the one I got using just a fuly connected network.

## Dirty Test Files
#### Test Audio 1
<audio src="data\test_x_01.wav" controls>Audio file not availablle</audio>
#### Test Audio 2
<audio src="data\test_x_02.wav" controls>Audio file not availablle</audio>
## Cleaned Test Files
#### Test Audio 1
<audio src="data\test_x_01_clean.wav" controls>Audio file not availablle</audio>
#### Test Audio 2
<audio src="data\test_x_02_clean.wav" controls>Audio file not availablle</audio>

In [14]:
# Deleting the model
del model

## TASK 2
In this task we are doing the same thing as above but instead of using 1D CNN we are going to use 2D CNN here. Since 2D CNN require 3D (height, width and channel of image) we must first convert the training and testing audio files to 3D images with one channel, such that 20 continous frames, which form a image, are used to predict the output of last frame in the image. Since there's not enough frames to get predictions for first 19 frames, we prepend a set of 19 silent frames, that is audio signal with values close to 0, to training and testing audio files.

In [16]:
# Loading the .wav files
# Ground Truth for training
s, sr=librosa.load(r'data\train_clean_male.wav', sr=None)
S=librosa.stft(s, n_fft=1024, hop_length=512)
# Unclean train file
sn, sr=librosa.load(r'data\train_dirty_male.wav', sr=None)
X=librosa.stft(sn, n_fft=1024, hop_length=512)
# Test 1
tn, sr=librosa.load(r'data\test_x_01.wav', sr=None)
A = librosa.stft(tn, n_fft=1024, hop_length=512)
# Test 2
t, sr=librosa.load(r'data\test_x_02.wav', sr=None)
B = librosa.stft(t, n_fft=1024, hop_length=512)

In [17]:
# Obtaining amplitude
y_data = np.abs(S).T
X_data = np.abs(X).T

# Test Data
X_test1 = np.abs(A).T
X_test2 = np.abs(B).T

In [18]:
# Adding silent frames to test data and obtaining 20x513 images from audio signal
silent_test1 = np.random.normal(0, 0.00000000000001, (19,513))
silent_test2 = np.random.normal(0, 0.00000000000001, (19,513))
silent_train = np.random.normal(0, 0.00000000000001, (19,513))

X_train = np.concatenate((silent_train, X_data))
X_test1 = np.concatenate((silent_test1, X_test1))
X_test2 = np.concatenate((silent_test2, X_test2))

train_images = np.stack([X_train[i:i+20] for i in range(len(X_train)-19)])[:,:,:,np.newaxis]
test1_images = np.stack([X_test1[i:i+20] for i in range(len(X_test1)-19)])[:,:,:,np.newaxis]
test2_images = np.stack([X_test2[i:i+20] for i in range(len(X_test2)-19)])[:,:,:,np.newaxis]

In [19]:
# Defining the model architecture
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape = (20,513,1), padding='same', activation='relu', kernel_initializer='he_normal'))
model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer='he_normal'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), padding = 'same', activation='relu', kernel_initializer='he_normal'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), padding = 'same', activation='relu', kernel_initializer='he_normal'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(2048, activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.3))
model.add(Dense(513, activation='relu', kernel_initializer='he_normal'))

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 20, 513, 32)       320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 513, 64)       18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 256, 64)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 256, 64)       36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 128, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 5, 128, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 64, 64)         0         
__________

In [21]:
model.compile(loss = 'mse', optimizer = Adam(), metrics = ['accuracy'])

In [22]:
history = model.fit(train_images, y_data, epochs = 100, batch_size = 128)

Epoch 1/100
2459/2459 [==============================] - 6s 2ms/step - loss: 0.1919 - acc: 0.0216
Epoch 2/100
2459/2459 [==============================] - 3s 1ms/step - loss: 0.0943 - acc: 0.0277
Epoch 3/100
2459/2459 [==============================] - 3s 1ms/step - loss: 0.0898 - acc: 0.0390
Epoch 4/100
2459/2459 [==============================] - 3s 1ms/step - loss: 0.0862 - acc: 0.0586
Epoch 5/100
2459/2459 [==============================] - 3s 1ms/step - loss: 0.0827 - acc: 0.0740
Epoch 6/100
2459/2459 [==============================] - 3s 1ms/step - loss: 0.0807 - acc: 0.0964
Epoch 7/100
2459/2459 [==============================] - 3s 1ms/step - loss: 0.0760 - acc: 0.0988
Epoch 8/100
2459/2459 [==============================] - 3s 1ms/step - loss: 0.0728 - acc: 0.1086
Epoch 9/100
2459/2459 [==============================] - 3s 1ms/step - loss: 0.0692 - acc: 0.1208
Epoch 10/100
2459/2459 [==============================] - 3s 1ms/step - loss: 0.0636 - acc: 0.1509
Epoch 11/100
2459/2

In [23]:
# Getting output prediction for test files and modified train file
outt = model.predict(train_images)
out1 = model.predict(test1_images)
out2 = model.predict(test2_images)

In [24]:
# Adding phase information
stestt = (X/np.abs(X))*outt.T
stest1 = (A/np.abs(A))*out1.T
stest2 = (B/np.abs(B))*out2.T

In [25]:
# SNR for training data
snr = 10*np.log10(np.add.reduce(np.square(S), axis = None)/np.add.reduce(np.square(S-stestt), axis = None))
print('SNR for training file:', np.abs(snr))

SNR for training file: 18.000774765102587


In [26]:
# Performing Inverse Short Time Fourier Transform to get the output signal
sh_test2 = librosa.istft(stest1, hop_length=512)
sh_test3 = librosa.istft(stest2, hop_length=512)

In [27]:
# Storing the cleaned test files
librosa.output.write_wav(r'data\test_x_03_clean.wav', sh_test2, sr)
librosa.output.write_wav(r'data\test_x_04_clean.wav', sh_test3, sr)

The 2D CNN isn't working as well as I expected it to. I tried **many** network architectures but still could not find satisfactory results. The noise of eating chips is lower in the output files and, similar to all the networks used before, the voice sounds computer generated and does not feel natural at all. There's also artifacts in the output speech signal similar to the output of all the networks used before.
Note: Sometimes the 2D CNN network gets stuck into local minima at the very beginning due to bad initializations (maybe!). Retraining the network fixes this!!

## Dirty Test Files
#### Test Audio 1
<audio src="data\test_x_01.wav" controls>Audio file not availablle</audio>
#### Test Audio 2
<audio src="data\test_x_02.wav" controls>Audio file not availablle</audio>
## Cleaned Test Files
#### Test Audio 1
<audio src="data\test_x_03_clean.wav" controls>Audio file not availablle</audio>
#### Test Audio 2
<audio src="data\test_x_04_clean.wav" controls>Audio file not availablle</audio>

In [28]:
# Deleting the model
del model